In [1]:
import os
import torch
import json
import pandas as pd
import re
import openai
from tqdm import tqdm
from openai import AzureOpenAI

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

GPU is not available


In [3]:
import csv
from google.colab import drive
drive.mount('/content/drive')

file_path='/content/drive/MyDrive/Colab Notebooks/comp4211 project/shuffled_A_B_relation_train.csv'
with open(file_path, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    csv_table = list(reader)

df = pd.DataFrame(csv_table)
df['predict']=0
df['EventB'] = df['EventB'].str.replace("[", "", regex=False).str.replace("]", "", regex=False).str.replace("'", "", regex=False).str.strip()
df.head(2)

Mounted at /content/drive


,EventA,EventB,relation,predict
0,PersonX causes trouble,"Is arrested, Thrown in jail",xEffect,0
1,PersonX drops off the radar,"has peace an quiet, gets more sleep",xEffect,0


In [4]:
folder_path = 'dataset'
#df = pd.DataFrame()
pange = pd.DataFrame()

prompt_loc = {
        "prototype": r"/content/drive/MyDrive/Colab Notebooks/comp4211 project/task2_prompt/task2_prompt.txt",
    }

client = AzureOpenAI(
    api_key="0ee412ce839d4a4a99e9abb02ee1eba7",                                              #
    api_version="2023-05-15",                                  #
    azure_endpoint="https://hkust.azure-api.net"
)

def simple_clean(sentence):
    return re.sub(r'[^\w\s]', '', sentence)
def default_clean(sentence):
    return sentence
def categorical_clean(sentence):
    categories = ["oEffect", "oReact", "oWant",  "xAttr","xEffect", "xIntent", "xNeed", "xReact", "xWant"]
    regex = f"{'|'.join(categories)}"
    return re.search(regex, sentence).group() if re.search(regex, sentence) else 'NA'

In [5]:
prompt_list = {name: open(prompt, "r").read() for name, prompt in prompt_loc.items()}

In [6]:
class GPT_QUERY:
    def __init__(self, set_size, table):
        self.set_size = set_size
        self.table = table

    def majority_vote(self, votes):
        votes_table = {}
        for vote in votes:
            if vote in votes_table:
                votes_table[vote] += 1
            else:
                votes_table[vote] = 1
        return max(votes_table, key=votes_table.get)

    def gpt(self, x, prompt, processor):
        response = client.chat.completions.create(
            model="gpt-35-turbo",
            messages=[
                {"role": "user", "content": prompt + x + "\nAnswer:"},
            ]
        )
        #print(prompt + x + "\nAnswer:")
        original = response.choices[0].message.content
        clean_sentence = processor(original)
        return clean_sentence

    def predict(self, prompt_type, processor=default_clean, vote=1):    ############# clean function
        set_size, table = self.set_size, self.table
        #prompt = prompt_loc[prompt_type]
        prompt = prompt_list[prompt_type]
        with tqdm(total=set_size) as bar:
            for index_A, row_A in table.head(set_size).iterrows():
                sentences = [row_A['EventA'], row_A['relation']]
                joined_sentence = " == ".join(sentences)
                try:
                    table.at[index_A, 'predict'] = self.majority_vote([ \
                        self.gpt(joined_sentence, prompt, processor) for _ in range(vote)])
                except Exception as error:
                    print(f"An error occurred at index {index_A}:", error)
                bar.update(1)

        table.head(set_size)[['predict', 'relation', 'EventA', 'EventB']].to_csv(f"{prompt_type}.csv")
        print(table.head(set_size)[['predict', 'relation']])



In [7]:
full_table=df

In [9]:
#prototype
prompt_list = {name: open(prompt, "r").read() for name, prompt in prompt_loc.items()}
gpt_client = GPT_QUERY(3, full_table)
gpt_client.predict("prototype", vote=3)


  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-6-8e13bef891da>:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'PersonX gets arrested.' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  table.at[index_A, 'predict'] = self.majority_vote([ \
100%|██████████| 3/3 [00:06<00:00,  2.31s/it]

                                     predict relation
0                     PersonX gets arrested.  xEffect
1  PersonX loses contact with their friends.  xEffect
2         PersonX gathers wood for the fire.    xNeed
